In [20]:
import cassio
import os
from getpass import getpass
from langchain_core.documents import Document


os.environ["ASTRA_DB_API_ENDPOINT"] = os.getenv("ASTRA_DB_API_ENDPOINT")
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vector_store = AstraDBVectorStore(
    collection_name="groq_db",
    embedding=embeddings,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT")
)



In [9]:
from datasets import load_dataset

philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
docs = []

for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

inserted_ids = vector_store.add_documents(docs)
print("inserted ids: ", len(inserted_ids))

In [12]:
texts = ["I think, therefore I am.", "To the things themselves!"]
metadatas = [{"author": "descartes"}, {"author": "husserl"}]
ids = ["desc_01", "huss_xy"]

inserted_ids_2 = vector_store.add_texts(texts=texts, metadatas=metadatas, ids=ids)
print("inserted ids: ", len(inserted_ids_2))

inserted ids:  2


In [13]:
results = vector_store.similarity_search("Our life is", k=5)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* as not-bound to life. [{'author': 'sartre'}]
* Life to the great majority is only a constant struggle for mere existence, with the certainty of losing it at last. [{'author': 'schopenhauer'}]
* We are what we are because we have been what we have been. [{'author': 'freud'}]
* To endure life remains, when all is said, the first duty of all living being Illusion can have no value if it makes this more difficult for us. [{'author': 'freud'}]
* In the blessings as well as in the ills of life, less depends upon what befalls us than upon the way in which it is met. [{'author': 'schopenhauer'}]


In [15]:
results = vector_store.max_marginal_relevance_search(
    "Our life is what we make of it",
    k=3,
    filter={"author": "aristotle"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* The quality of life is determined by its activities. [{'author': 'aristotle'}]
* Love is composed of a single soul inhabiting two bodies. [{'author': 'aristotle'}]
* We must be neither cowardly nor rash but courageous. [{'author': 'aristotle'}]


In [16]:
delete = vector_store.delete(inserted_ids[:3])
print(delete)

True


In [17]:
!curl -L \
    "https://github.com/awesome-astra/datasets/blob/main/demo-resources/what-is-philosophy/what-is-philosophy.pdf?raw=true" \
    -o "what-is-philosophy.pdf"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 55220  100 55220    0     0  40082      0  0:00:01  0:00:01 --:--:-- 40082


In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


retriever = vector_store.as_retriever(search_kwargs={"k": 3})

template = """
You are a philosopher who have extensive expereince in philosophy and draws inspiration from the philosophers of the past and the present to craft well answers to questions.
Use the provided context as the basisi for your answer, don't try to make up new reasoning paths. Your answers must be concise and to the point.

Context: {context}

Question: {question}

Helpful Answers: 
"""

philo_prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | philo_prompt
    | llm
    | StrOutputParser()
)

In [22]:
chain.invoke("How does define security blanket")

'A security blanket can be defined as a source of comfort and protection, often providing a sense of safety and familiarity in challenging or uncertain situations.'

In [23]:
vector_store.delete_collection()